In [ ]:
import pandas as pd
import os

In [ ]:
# Train.csv
from google.colab import files
files.upload()

In [ ]:
!ls -al

In [ ]:
annotation = pd.read_csv('Train.csv')

In [ ]:
Prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
Danger = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Mandatory = [33, 34, 35, 36, 37, 38, 39, 40]
Ohter = [6, 12, 13, 14, 17, 32, 41, 42]

categories = ['Prohibitory', 'Danger', 'Mandatory', 'Ohter']

In [ ]:
annotation['CategoryID'] = 0

In [ ]:
annotation.loc[annotation['ClassId'].isin(Prohibitory),'CategoryID'] = 0
annotation.loc[annotation['ClassId'].isin(Danger),'CategoryID'] = 1
annotation.loc[annotation['ClassId'].isin(Mandatory),'CategoryID'] = 2
annotation.loc[annotation['ClassId'].isin(Ohter),'CategoryID'] = 3

In [ ]:
annotation['centerX'] = ((annotation['Roi.X1']+annotation['Roi.X2'])/2) / annotation['Width']
annotation['centerY'] = ((annotation['Roi.Y1']+annotation['Roi.Y2'])/2) / annotation['Height']

In [ ]:
annotation['width'] = (annotation['Roi.X2'] - annotation['Roi.X1']) / annotation['Width']
annotation['height'] = (annotation['Roi.Y2'] - annotation['Roi.Y1']) / annotation['Height']

In [ ]:
annotation = annotation.drop(['Width', 'Height', 'Roi.X1','Roi.Y1','Roi.X2','Roi.Y2','ClassId'], axis='columns')

In [ ]:
for index, data in annotation.iterrows():
    print(index, data['Path'])

In [ ]:
with open('classes.names', 'w') as names:
    for category in categories:
        line = category + '\n'
        names.write(line)
    print('[classes.names] is created') 

In [ ]:
!ls -al

In [ ]:
from google.colab import files
files.download('classes.names')

In [ ]:
with open('traffic.data', 'w') as data:
    data.write('classes = ' + str(len(categories)) + '\n')
    data.write('train = ' +'train.txt' + '\n')
    data.write('valid = ' +'test.txt' + '\n')
    data.write('names = ' +'classes.names' + '\n')
    data.write('backup = backup')
    print('[custom_data.data] is created')

In [ ]:
!mkdir backup

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!ls -al '/gdrive/MyDrive/darknet'

In [ ]:
!cp -r /gdrive/MyDrive/darknet/Train.zip .

In [ ]:
!ls -al

In [ ]:
annotation.head()

In [ ]:
!unzip Train.zip

In [ ]:
paths = []
for index, data in annotation.iterrows():
    # print(index, data['Path'])
    txt_file = data['Path'].split('.')[0] + '.txt'
    line = str(data['CategoryID']) + ' ' + str(data['centerX']) + ' ' + str(data['centerY']) + ' ' + str(data['width']) + ' ' + str(data['height']) + '\n'
    with open(txt_file, 'w') as names:
        names.write(line)
    image_path = data['Path'] + '\n'
    paths.append(image_path)
    print(index, txt_file, line)

In [ ]:
paths

In [ ]:
test_percentage = 0.2

In [ ]:
num_test = int(len(paths) * test_percentage)
print(num_test)

In [ ]:
paths_test = paths[:num_test]
paths_train = paths[num_test:]

In [ ]:
with open('train.txt', 'w') as train_txt:
    for path in paths_train:
        train_txt.write(path)
    print('[train.txt] is created')

In [ ]:
with open('test.txt', 'w') as test_txt:
    for path in paths_test:
        test_txt.write(path)
    print('[test.txt] is created')

In [ ]:
!ls -al

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!ls -al

In [ ]:
!chmod +x ./darknet

In [ ]:
#Train the Model
!./darknet detector train traffic.data traffic-train-yolo.cfg darknet53.conv.74 -dont_show